# Consolidating Route Patterns in GTFS

Occurs after running R script that filters the trips, schedules, etc. by a chosen date.
This part first came after an import to TransCAD but this is an alternate approach of consolidating solely based on GTFS instead of trying to join the calculated trips per Route_Name back to the route_pattern_id. There isn't a great field to join back on as we don't know exactly how TransCAD is translating the GTFS information into the Route_Names and how the routes are being consolidated as the number of unique route patterns differs between TransCAD and GTFS with GTFS having less unique route patterns in its trip table *(both post R filtering)

As discussed with Marty and Sabiheh, the goal of this consolidation step is to re-assign route patterns with low numbers of trips across the day ( less than three in each time period) to route_pattern_ids with the most trips within their matching Route, Direction, and Headsign. This gets filtered down to the trip level where if a trip's route pattern is consolidated, it will be replaced with the route_pattern_id with max trips.

All trips with same headsign/route/direction  are assigned the same route_pattern_id. The route_pattern_id that represents the group is the route_pattern_id with the most trips per day.

If less than 3 trips per tod period across all day, assign the route_pattern_id of the dir/route group with the most trips.

Calc TOD by 3-3 not 6:30 - 30:29

reorder stop times at end by trip_id

In [89]:
import time
import datetime

import matplotlib
matplotlib.use('agg')  # allows notebook to be tested in Travis

import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import pandana as pdna
import time

import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network

from tqdm import tqdm

%matplotlib inline

In [90]:
# required bbox including all of Massachusetts and RI as well as parts of NH, CT, NY
bbox = (-73.7207, 41.1198, -69.7876, 43.1161)
# path to the downloaded and cleaned gtfs - mbta recap file for fall 2018
#   this could also be a folder of gtfs folders (pre merge of multiple gtfs)

path_to_gtfs = r"C:\Users\matkinson.AD\Downloads\Dec07_Sandbox\Part_2_GTFS_R\mbta2018_102418_20221207"

In [91]:
loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(gtfsfeed_path= path_to_gtfs,
                                           validation=True,
                                           verbose=True,
                                           bbox=bbox,
                                           remove_stops_outsidebbox=False,
                                           append_definitions=True)

Checking GTFS text file header whitespace... Reading files using encoding: utf-8 set in configuration.
GTFS text file header whitespace check completed. Took 0.19 seconds
--------------------------------
Processing GTFS feed: mbta2018_102418_20221207
The unique agency id: mbta was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.03 seconds
Unique GTFS feed id operation complete. Took 0.00 seconds
No GTFS feed stops were found to be outside the bounding box coordinates
mbta2018_102418_20221207 GTFS feed stops: coordinates are in northwest hemisphere. Latitude = North (90); Longitude = West (-90).
Appended route type to stops
Appended route type to stop_times
--------------------------------
Added descriptive definitions to stops, routes, stop_times, and trips tables
Successfully converted ['departure_time'] to seconds past midnight and appended new columns to stop_times. Took 1.43 seconds
1 GTFS feed file(s) successfully read as 

Needs/Steps:
- Number of trips per time period per route_pattern_id
    - Midpoint time of each trip
    - Each trip classified by TOD (based on midpoint)
    - Sum of trips per TOD by route_pattern_id
- route_pattern_id with most daily trips per Route, Direction, Headsign
    - Sum all tod trips per route_pattern_id
    - grab just the max per Route, Direction, Headsign (but keep route_pattern_id)
- consolidate route patterns by Route, Direction, Headsign
    - if route_pattern_id has less than 3 trips in each of the 4 TODs, replace with max trips route_pattern_id

In [92]:
non_bus_routes = list(gtfsfeeds_dfs.routes.query('route_type != 3').route_id.unique())
non_bus_trips = gtfsfeeds_dfs.trips.query('route_id in @non_bus_routes')
bus_trips = gtfsfeeds_dfs.trips.query('route_id not in @non_bus_routes')
bus_trips

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed,unique_agency_id,unique_feed_id,bikes_allowed_desc,wheelchair_accessible_desc
0,1,BUS42018-hbc48wk1-Weekday-02,38230147,Harvard,NaN,0,C01-21,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...
1,1,BUS42018-hbc48wk1-Weekday-02,38230148,Harvard,NaN,0,C01-21,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...
2,1,BUS42018-hbc48wk1-Weekday-02,38230154,Harvard,NaN,0,C01-16,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...
3,1,BUS42018-hbc48wk1-Weekday-02,38230155,Harvard,NaN,0,C01-6,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...
4,1,BUS42018-hbc48wk1-Weekday-02,38230157,Harvard,NaN,0,C01-20,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14257,99,BUS42018-hbg48011-Weekday-02,38443852,Wellington,NaN,1,G99-57,990056,1,NaN,99-7-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...
14258,99,BUS42018-hbg48011-Weekday-02,38443854,Wellington,NaN,1,G99-57,990056,1,NaN,99-7-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...
14259,99,BUS42018-hbg48011-Weekday-02,38443856,Wellington,NaN,1,G101-80,990056,1,NaN,99-7-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...
14260,99,BUS42018-hbg48011-Weekday-02,38443858,Wellington,NaN,1,G99-58,990056,1,NaN,99-7-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...


In [93]:
def get_start_stop_times(stop_times):    
    '''for every trip, grab the start time and stop time of the trip'''
    chocula =0 
    for trip_id in stop_times['trip_id'].unique():
        max_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.max()')[['trip_id','arrival_time']]
        min_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.min()')[['trip_id','arrival_time']]
        r2 = min_row.merge(max_row, how='left', on='trip_id', suffixes = ('_start','_end'))
        if chocula == 0:
            flintstone = pd.DataFrame(r2)
        else:
            flintstone=pd.concat([flintstone,r2])
        chocula +=1
    return(flintstone)


In [94]:
simpson = get_start_stop_times(gtfsfeeds_dfs.stop_times)

Note: the MBTA service day is 3AM - 26:59. For TOD assignment, everything that isn't between the hours of 6:30 - 19:00 is counted as NT. This addresses this issue as it includes 19:00-26:39 AND 3:00 - 6:30.

In [95]:
def assign_tod(start_stop):
    '''calculate midpoint of trip, use midpoint to assign TOD'''
    
    start_stop['at_end_dec'] = (
        (
            (start_stop['arrival_time_end'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_end'].str.split(":").str[1]
            ).astype('int32')/60)))
    start_stop['at_start_dec'] = (
        (
            (start_stop['arrival_time_start'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_start'].str.split(":").str[1]
            ).astype('int32')/60)))
    
    start_stop['midpoint'] = start_stop['at_start_dec'] + ((start_stop['at_end_dec']-start_stop['at_start_dec'])/2)
    start_stop['tod'] = np.where(start_stop['midpoint'].between(6.50,9.50),'AM', np.where(
        start_stop['midpoint'].between(9.50,15.00), 'MD', np.where(
            start_stop['midpoint'].between(15.00,19.00),'PM', 'NT' 
        )
            ) 
        )
    
    return start_stop


In [96]:
smurf = assign_tod(simpson)

In [97]:
smurf

,trip_id,arrival_time_start,arrival_time_end,at_end_dec,at_start_dec,midpoint,tod
0,37940074,19:25:00,20:03:00,20.050000,19.416667,19.733333,NT
0,37940075,06:35:00,07:03:00,7.050000,6.583333,6.816667,AM
0,37940076,07:14:00,07:53:00,7.883333,7.233333,7.558333,AM
0,37940077,11:12:00,11:50:00,11.833333,11.200000,11.516667,MD
0,37940079,05:48:00,06:25:00,6.416667,5.800000,6.108333,NT
...,...,...,...,...,...,...,...
0,CR-Weekday-Fall-18-924,20:35:00,21:08:00,21.133333,20.583333,20.858333,NT
0,CR-Weekday-Fall-18-925,21:40:00,22:19:00,22.316667,21.666667,21.991667,NT
0,CR-Weekday-Fall-18-926,22:35:00,23:10:00,23.166667,22.583333,22.875000,NT
0,CR-Weekday-Fall-18-927,22:40:00,23:20:00,23.333333,22.666667,23.000000,NT


#### Update the route_pattern_ids
- Getting duplicate ids where same number of trips for max, so choosing one arbitrarily.
- The section can be expanded to try to determine which has the most trips during peak period, but for now this works.

In [98]:
# FOR R/H/D, select RPID with most trips

    # add TOD to trip table
tod_trips = bus_trips.merge(smurf[['trip_id','tod']], how='left', on='trip_id')

    # get the number of trips per R/H/D & rpid
day_rpid = tod_trips.groupby(by=['route_id','trip_headsign','direction_id','route_pattern_id']).agg({'trip_id':'nunique'})
day_rpid = day_rpid.rename(columns = {'trip_id':'daily_trips'}).reset_index()

    # for each R/H/D, select just the route_pattern_id with the most daily trips
max_rpid = day_rpid.groupby(by=['route_id','trip_headsign','direction_id']).apply(lambda g: g[g['daily_trips'] == g['daily_trips'].max()])[['route_pattern_id','daily_trips']].reset_index()
max_rpid = max_rpid[['route_id','trip_headsign','direction_id','route_pattern_id']].rename(columns = {'route_pattern_id':'route_pattern_id_new'})

# because there are several cases where the max is held by two different route_pattern_ids, chose one arbitrarily
max_rpid = max_rpid.drop_duplicates(subset=['route_id','trip_headsign','direction_id'])
max_rpid

,route_id,trip_headsign,direction_id,route_pattern_id_new
0,1,Dudley,1,1-_-1
1,1,Harvard,0,1-_-0
2,10,Andrew via South Bay Center,0,10-6-0
3,10,City Point,0,10-_-0
4,10,City Point via South Bay Center,0,10-9-0
...,...,...,...,...
642,97,Malden,0,97-5-0
643,97,Wellington,1,97-5-1
644,99,Malden,1,99-1-1
645,99,Wellington,1,99-7-1


In [99]:
# join trip table with the RPID with most trips (join is on R/H/D, RPID is an attribute of the table)

trips_update1 = bus_trips.merge(max_rpid, how='left', on=['route_id','trip_headsign','direction_id'])
# update the route_pattern_ids based on R/H/D max
    # this may be the same as original RPID as this should cover all R/H/D combos (therefore all trips)
trips_update1['route_pattern_id'] = trips_update1['route_pattern_id_new']
trips_update1 = trips_update1[gtfsfeeds_dfs.trips.columns]

# check to see if there are any duplicated trip_ids (meaning they would have multiple RPIDs, possibly from multiple max trip RPIDs)
trips_update1[trips_update1.duplicated(subset=['trip_id'], keep =False)]

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed,unique_agency_id,unique_feed_id,bikes_allowed_desc,wheelchair_accessible_desc


#### Now - update route/headsign/direction combos where each TOD period has less than three trips 
- take the route_pattern_id with max number of daily trips per route/direction

In [100]:
trips_update1.groupby(by=['route_pattern_id','route_id','direction_id','trip_headsign']).agg({'trip_id':'count','service_id':'nunique'})

,,,,trip_id,service_id
route_pattern_id,route_id,direction_id,trip_headsign,,
1-_-0,1,0,Harvard,114,1
1-_-1,1,1,Dudley,109,1
10-2-1,10,1,Townsend & Humboldt,1,1
10-6-0,10,0,Andrew via South Bay Center,1,1
10-9-0,10,0,City Point via South Bay Center,31,1
...,...,...,...,...,...
97-5-0,97,0,Malden,16,1
97-5-1,97,1,Wellington,16,1
99-1-1,99,1,Malden,1,1


In [101]:
# add tod back to trips table
tod_trips2 = trips_update1.merge(smurf[['trip_id','tod']], how='left', on='trip_id')

# for each TOD and R/H/D
tod_rpid2 = tod_trips2.groupby(by=['route_id','trip_headsign','direction_id','tod']).agg({'tod':'count','route_pattern_id':'nunique','service_id':'nunique'})
tod_rpid2 = tod_rpid2.rename(columns = {'tod':'trips_per_tod'}).reset_index()

In [102]:
# select just the trips that need to be updated
tod_rpid3 = tod_rpid2[['route_id','trip_headsign','direction_id','tod','trips_per_tod']]
    # for R/H/D, get trips_per_tod separated into columns by TOD
woolf = tod_rpid3.pivot_table(index = ['route_id','trip_headsign','direction_id'], columns = ['tod'])

    # get just the trips where all 4 periods have less than 3 trips
needs_update = woolf['trips_per_tod'].reset_index().fillna(0).query('AM < 3 & MD < 3 & PM < 3 & NT < 3')

needs_update

tod,route_id,trip_headsign,direction_id,AM,MD,NT,PM
2,10,Andrew via South Bay Center,0,1.0,0.0,0.0,0.0
7,10,Townsend & Humboldt,1,1.0,0.0,0.0,0.0
9,100,Fellsway Garage via Fellsway,1,0.0,0.0,0.0,1.0
12,101,Medford Square,0,1.0,0.0,1.0,0.0
20,106,Franklin Square via Lebanon Loop,0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
603,9,Kenmore via Boston Latin,1,1.0,0.0,0.0,0.0
619,95,Fellsway Garage via High St,1,0.0,0.0,0.0,2.0
620,95,Fellsway Garage via Winthrop St,1,0.0,2.0,0.0,0.0
623,95,West Medford via High St,0,0.0,0.0,1.0,0.0


In [103]:
# get the number of trips per R/D & rpid
day_rpid2 = tod_trips2.groupby(by=['route_id','direction_id','route_pattern_id']).agg({'trip_id':'nunique'})
day_rpid2 = day_rpid2.rename(columns = {'trip_id':'daily_trips'}).reset_index()

    # for each R/D, select just the route_pattern_id with the most daily trips
max_rpid2 = day_rpid2.groupby(by=['route_id','direction_id']).apply(lambda g: g[g['daily_trips'] == g['daily_trips'].max()])[['route_pattern_id','daily_trips']].reset_index()
max_rpid2 = max_rpid2[['route_id','direction_id','route_pattern_id']].rename(columns = {'route_pattern_id':'route_pattern_id_new'})

# because there are several cases where the max is held by two different route_pattern_ids, chose one arbitrarily
max_rpid2 = max_rpid2.drop_duplicates(subset=['route_id','direction_id'])

max_rpid2

,route_id,direction_id,route_pattern_id_new
0,1,0,1-_-0
1,1,1,1-_-1
2,10,0,10-9-0
3,10,1,10-9-1
4,100,0,100-3-0
...,...,...,...
358,96,1,96-_-1
359,97,0,97-5-0
360,97,1,97-5-1
361,99,0,99-7-0


In [104]:
# update the route_pattern_ids for just R/H/D combos that have < 3 trips in each of the 4 TOD periods
update_table = needs_update.merge(max_rpid2, how='left', on=['route_id','direction_id'])

update_table = update_table.drop_duplicates(subset=['route_id','direction_id','trip_headsign'])

update_table

,route_id,trip_headsign,direction_id,AM,MD,NT,PM,route_pattern_id_new
0,10,Andrew via South Bay Center,0,1.0,0.0,0.0,0.0,10-9-0
1,10,Townsend & Humboldt,1,1.0,0.0,0.0,0.0,10-9-1
2,100,Fellsway Garage via Fellsway,1,0.0,0.0,0.0,1.0,100-3-1
3,101,Medford Square,0,1.0,0.0,1.0,0.0,101-3-0
4,106,Franklin Square via Lebanon Loop,0,1.0,0.0,0.0,0.0,106-_-0
...,...,...,...,...,...,...,...,...
157,9,Kenmore via Boston Latin,1,1.0,0.0,0.0,0.0,9-_-1
158,95,Fellsway Garage via High St,1,0.0,0.0,0.0,2.0,95-_-1
159,95,Fellsway Garage via Winthrop St,1,0.0,2.0,0.0,0.0,95-_-1
160,95,West Medford via High St,0,0.0,0.0,1.0,0.0,95-_-0


#### Update those route_pattern_ids!

In [105]:
trips_update3 = trips_update1.merge(
    update_table[['route_id','direction_id','trip_headsign','route_pattern_id_new']], 
    how='left',on=['route_id','direction_id','trip_headsign'])

trips_update3['route_pattern_id'] = np.where(
    (
        (trips_update3['route_pattern_id_new'].isna())), 
    trips_update3['route_pattern_id'], 
    trips_update3['route_pattern_id_new'])

In [106]:
# update headsign for all trips with updated route_pattern_id
rpid_th = trips_update3.groupby(by=['route_pattern_id','trip_headsign','block_id','shape_id','service_id']).agg({'trip_id':'count'}).reset_index()
rpid_th_max = rpid_th.groupby(by=['route_pattern_id']).apply(lambda g: g[g['trip_id'] == g['trip_id'].max()]).reset_index(drop=True)
rpid_th_max = rpid_th_max.drop_duplicates(subset='route_pattern_id')
trips_update4 = trips_update3.merge(
    rpid_th_max[['route_pattern_id','trip_headsign','block_id','shape_id','service_id']], 
    how='left',on=['route_pattern_id'], suffixes=(None, '_to_rplce'))
trips_update4['trip_headsign'] = trips_update4['trip_headsign_to_rplce']
trips_update4['block_id'] = trips_update4['block_id_to_rplce']
#trips_update4['shape_id'] = trips_update4['shape_id_to_rplce']
trips_update4['service_id'] = trips_update4['service_id_to_rplce']
trips_update4

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,...,bikes_allowed,unique_agency_id,unique_feed_id,bikes_allowed_desc,wheelchair_accessible_desc,route_pattern_id_new,trip_headsign_to_rplce,block_id_to_rplce,shape_id_to_rplce,service_id_to_rplce
0,1,BUS42018-hbc48wk1-Weekday-02,38230147,Harvard,NaN,0,C01-1,010038,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard,C01-1,010038,BUS42018-hbc48wk1-Weekday-02
1,1,BUS42018-hbc48wk1-Weekday-02,38230148,Harvard,NaN,0,C01-1,010038,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard,C01-1,010038,BUS42018-hbc48wk1-Weekday-02
2,1,BUS42018-hbc48wk1-Weekday-02,38230154,Harvard,NaN,0,C01-1,010038,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard,C01-1,010038,BUS42018-hbc48wk1-Weekday-02
3,1,BUS42018-hbc48wk1-Weekday-02,38230155,Harvard,NaN,0,C01-1,010038,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard,C01-1,010038,BUS42018-hbc48wk1-Weekday-02
4,1,BUS42018-hbc48wk1-Weekday-02,38230157,Harvard,NaN,0,C01-1,010038,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard,C01-1,010038,BUS42018-hbc48wk1-Weekday-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14257,99,BUS42018-hbf48011-Weekday-02,38443852,Wellington,NaN,1,F99-36,990056,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Wellington,F99-36,990056,BUS42018-hbf48011-Weekday-02
14258,99,BUS42018-hbf48011-Weekday-02,38443854,Wellington,NaN,1,F99-36,990056,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Wellington,F99-36,990056,BUS42018-hbf48011-Weekday-02
14259,99,BUS42018-hbf48011-Weekday-02,38443856,Wellington,NaN,1,F99-36,990056,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Wellington,F99-36,990056,BUS42018-hbf48011-Weekday-02
14260,99,BUS42018-hbf48011-Weekday-02,38443858,Wellington,NaN,1,F99-36,990056,1,NaN,...,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Wellington,F99-36,990056,BUS42018-hbf48011-Weekday-02


#### CHECKS

In [107]:
trips_update3[trips_update3['trip_id'].duplicated(keep=False)]

len(trips_update4['trip_id'].unique())
len(trips_update4['route_pattern_id'].unique())
len(bus_trips['route_pattern_id'].unique())


866

In [108]:
trips_update5 = trips_update4[gtfsfeeds_dfs.trips.columns].merge(bus_trips[['trip_id','route_pattern_id','trip_headsign']], how='left', on='trip_id', suffixes=(None,'_old'))

In [109]:
trips_update5.query('route_pattern_id=="217-9-1"')

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed,unique_agency_id,unique_feed_id,bikes_allowed_desc,wheelchair_accessible_desc,route_pattern_id_old,trip_headsign_old
3321,217,BUS42018-hbq48011-Weekday-02,38304006,Ashmont,NaN,1,Q216-40,2170039,1,NaN,217-9-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,217-2-1,Bryant Ave & Upton St
3322,217,BUS42018-hbq48011-Weekday-02,38304005,Ashmont,NaN,1,Q216-40,2170072,1,NaN,217-9-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,217-3-1,Ashmont
3323,217,BUS42018-hbq48011-Weekday-02,38303753,Ashmont,NaN,1,Q216-40,2170069,1,NaN,217-9-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,217-9-1,Ashmont
3324,217,BUS42018-hbq48011-Weekday-02,38303755,Ashmont,NaN,1,Q216-40,2170069,1,NaN,217-9-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,217-9-1,Ashmont
3325,217,BUS42018-hbq48011-Weekday-02,38303756,Ashmont,NaN,1,Q216-40,2170069,1,NaN,217-9-1,1,mbta,mbta2018_102418_20221207_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,217-9-1,Ashmont


In [110]:
len(trips_update5.query('route_pattern_id != route_pattern_id_old').query('trip_headsign == trip_headsign_old'))


1440

#### Replace Trips Table

In [111]:
trips_keep_safe = gtfsfeeds_dfs.trips

In [112]:
#gtfsfeeds_dfs.trips = pd.concat([trips_update5,non_bus_trips])
# keep trips just busses for next part too
gtfsfeeds_dfs.trips = trips_update5

### Create Generic Stop Times & Stop Sequence per Route Pattern

- Can't just change route_pattern_id as TransCAD does not use this field to combine trips into routes. There is no effect on the import.
- Working theory is that to consolidate routes one must use the stop_times.txt table as it defines the stop sequence for every trip. Theoretically, this is being used to consolidate trips into routes based on whether they have the same stop sequence.

Plan:
- Explore if stop times differ depending on TOD or if only realtime GTFS takes into account traffic.
    - For every route_pattern_id, get the average trip length (in minutes).
- For every route_pattern_id, get the average number of minutes between each pair of stops in the stop sequence.
- Replace the stop times and sequence for trips that had their route_pattern_id updated with the generic stop sequence and times per route_pattern_id created in the previous step. 
    - Keep the start time and work off of that.
    - Arrival time will equal departure time given that the difference is usually less than a minute. Will assume difference in time can be included in the minutes to next arrival time for aggregate modeling purposes.

In [113]:
# REDO
# flag the first stop in every trip
gtfsfeeds_dfs.stop_times['first_stop'] =  0
gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.groupby('trip_id').stop_sequence.idxmin(),'first_stop'] = 1

#chocula = 0
rpid_stop_dict = {}
# for each route_pattern_id calculate the most common stop pattern
for rpid in tqdm(gtfsfeeds_dfs.trips['route_pattern_id'].unique()):
    patterns = []
    rpid_trips = gtfsfeeds_dfs.trips.query('route_pattern_id == @rpid')['trip_id'].to_list()
    rpid_stops = gtfsfeeds_dfs.stop_times.query('@rpid_trips in trip_id').sort_values('stop_sequence')
    
    # for every trip, get stop pattern and save into a list of lists
    for tid in rpid_stops['trip_id']:
        patterns.append([tid,':'.join(rpid_stops.query('trip_id == @tid')['stop_id'].to_list())])
    
    # make data frame of every trip and its pattern of stops
    df = pd.DataFrame(patterns, columns = ['trip_id','stop_pattern'])
    # get most common stop_pattern
    df_grby = df.groupby('stop_pattern').count().reset_index()
    max_stop_pattern = df_grby.query('trip_id == trip_id.max()')['stop_pattern'].to_list()
    # get all the trip_ids that represent the route_pattern_id's most popular stop pattern
    rep_trips = df.query('stop_pattern == @max_stop_pattern[0]')['trip_id'].to_list()

    rpid_stop_dict[rpid] = rep_trips
    #chocula += 1
    #if chocula > 0:
        #break

100%|██████████| 489/489 [20:48<00:00,  2.55s/it]


In [114]:
# REDO PART 2
# for every row where the route_pattern_id has been updated & doesn't have the majority stop pattern
trip_stop_replace = {}
trip_shape_replace = {}
for idx, row in tqdm(gtfsfeeds_dfs.trips.iterrows()):
    # identify trip_id
    tid = row['trip_id']
    # get the route_pattern_id for the trip
    rpid = row['route_pattern_id']

    if (tid not in rpid_stop_dict[rpid]): # if trip_id is not in the majority stop pattern
        # get all of the trips associated with that route_pattern_id (and stop sequence pattern)
        all_trips = rpid_stop_dict[rpid]

        # get the start time (for first stop) for trip_id
        start_time = gtfsfeeds_dfs.stop_times.query('(trip_id == @tid) & (first_stop == 1)')
        # get the start time (for first stop) for all trips that share same route_pattern_id
        all_start_times = gtfsfeeds_dfs.stop_times.query('(trip_id in @all_trips) & (first_stop == 1)')

        # create list getting the difference in start times between the selected trip and all the other trips that share the same route_pattern_id
        test_list = [[x,(abs(start_time['departure_time_sec']- all_start_times.query('trip_id == @x'))['departure_time_sec'].iloc[0])] for x in all_start_times['trip_id']]
        close = {}
        close = {sub[0]:sub[1] for sub in test_list}
            
        # get trip with the minimum difference in start time    
        min_t = min(close, key=close.get)
        if tid != min_t:
            trip_stop_replace[tid] = min_t
            trip_shape_replace[tid] = gtfsfeeds_dfs.trips.query('trip_id == @min_t')['shape_id'].values[0]
    else:
        trip_shape_replace[tid] = gtfsfeeds_dfs.trips.query('trip_id == @tid')['shape_id'].values[0]

14262it [10:07, 23.46it/s] 


In [115]:
trip_shape_replace_tab = pd.DataFrame.from_dict(
    trip_shape_replace, orient='index').reset_index().rename(
        columns = {'index':'trip_id', 0:'shape_id_update'})

trip_shape_replace_tab

,trip_id,shape_id_update
0,38230147,010038
1,38230148,010038
2,38230154,010038
3,38230155,010038
4,38230157,010038
...,...,...
14257,38443852,990056
14258,38443854,990056
14259,38443856,990056
14260,38443858,990056


In [116]:
gtfsfeeds_dfs.trips['shape_id_new'] = gtfsfeeds_dfs.trips['trip_id'].apply(lambda x: trip_shape_replace[x])
#gtfsfeeds_dfs.trips.query('shape_id_new != shape_id')

gtfsfeeds_dfs.trips['shape_id'] = gtfsfeeds_dfs.trips['shape_id_new']



In [117]:
gtfsfeeds_dfs.stop_times = gtfsfeeds_dfs.stop_times.sort_values('stop_sequence')
# for every trip (in dictionary with value = trip sharing route_pattern_id with closest start time)
for tid in tqdm(trip_stop_replace.values()):
    # update value of time_between_stops for every stop in the selected trip
        # time between stops calculated by subtracting the prior departure_time_sec from current (why stop_sequence order is important)
    gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.loc[:,'trip_id'] == str(tid), 'time_between_stops'] = gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.loc[:,'trip_id'] == str(tid), 'departure_time_sec'].diff()

gtfsfeeds_dfs.stop_times

100%|██████████| 1671/1671 [02:00<00:00, 13.85it/s]


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint,checkpoint_id,unique_agency_id,unique_feed_id,route_type,pickup_type_desc,drop_off_type_desc,timepoint_desc,departure_time_sec,first_stop,time_between_stops
470720,Boat-F4-0715-Long-Weekday,07:15:00,07:15:00,Boat-Long-South,0,NaN,0,1,1,NaN,mbta,mbta2018_102418_20221207_1,4,Regularly Scheduled,Not available,Exact times,26100,1,NaN
470810,Boat-F4-1615-Charlestown-Weekday,16:15:00,16:15:00,Boat-Charlestown,0,NaN,0,1,1,NaN,mbta,mbta2018_102418_20221207_1,4,Regularly Scheduled,Not available,Exact times,58500,1,NaN
470605,Boat-F1-1300-Hingham-Weekday,13:00:00,13:00:00,Boat-Hingham,0,NaN,0,1,1,NaN,mbta,mbta2018_102418_20221207_1,4,Regularly Scheduled,Not available,Exact times,46800,1,NaN
470808,Boat-F4-1600-Long-Weekday,16:00:00,16:00:00,Boat-Long-South,0,NaN,0,1,1,NaN,mbta,mbta2018_102418_20221207_1,4,Regularly Scheduled,Not available,Exact times,57600,1,NaN
470806,Boat-F4-1600-Charlestown-Weekday,16:00:00,16:00:00,Boat-Charlestown,0,NaN,0,1,1,NaN,mbta,mbta2018_102418_20221207_1,4,Regularly Scheduled,Not available,Exact times,57600,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28690,38091940,16:58:00,16:58:00,70260,680,NaN,1,0,0,NaN,mbta,mbta2018_102418_20221207_1,0,Not available,Regularly Scheduled,Approximate times,61080,0,NaN
27410,38091876,07:33:00,07:33:00,70260,680,NaN,1,0,0,NaN,mbta,mbta2018_102418_20221207_1,0,Not available,Regularly Scheduled,Approximate times,27180,0,NaN
30150,38092013,21:56:00,21:56:00,70260,680,NaN,1,0,0,NaN,mbta,mbta2018_102418_20221207_1,0,Not available,Regularly Scheduled,Approximate times,78960,0,NaN
43830,38093911,12:51:00,12:51:00,70260,680,NaN,1,0,0,NaN,mbta,mbta2018_102418_20221207_1,0,Not available,Regularly Scheduled,Approximate times,46260,0,NaN


In [118]:
gtfsfeeds_dfs.stop_times.query('time_between_stops > 0')

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint,checkpoint_id,unique_agency_id,unique_feed_id,route_type,pickup_type_desc,drop_off_type_desc,timepoint_desc,departure_time_sec,first_stop,time_between_stops
325375,38406605,04:27:00,04:27:00,15782,2,NaN,0,0,0,NaN,mbta,mbta2018_102418_20221207_1,3,Regularly Scheduled,Regularly Scheduled,Approximate times,16020,0,120.0
348376,38428815,06:02:00,06:02:00,15059,2,NaN,0,0,0,NaN,mbta,mbta2018_102418_20221207_1,3,Regularly Scheduled,Regularly Scheduled,Approximate times,21720,0,120.0
73757,38173969,18:12:00,18:12:00,16535,2,NaN,0,0,0,NaN,mbta,mbta2018_102418_20221207_1,3,Regularly Scheduled,Regularly Scheduled,Approximate times,65520,0,120.0
73280,38173946,12:11:00,12:11:00,902,2,NaN,0,0,0,NaN,mbta,mbta2018_102418_20221207_1,3,Regularly Scheduled,Regularly Scheduled,Approximate times,43860,0,60.0
346418,38428746,07:01:00,07:01:00,9318,2,NaN,0,0,0,NaN,mbta,mbta2018_102418_20221207_1,3,Regularly Scheduled,Regularly Scheduled,Approximate times,25260,0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404817,38485899,09:57:00,09:57:00,641,90,NaN,0,0,0,NaN,mbta,mbta2018_102418_20221207_1,3,Regularly Scheduled,Regularly Scheduled,Approximate times,35820,0,60.0
404640,38485897,07:57:00,07:57:00,642,90,NaN,0,0,0,NaN,mbta,mbta2018_102418_20221207_1,3,Regularly Scheduled,Regularly Scheduled,Approximate times,28620,0,60.0
182540,38261803,18:22:00,18:22:00,8921,90,NaN,1,0,0,linsh,mbta,mbta2018_102418_20221207_1,3,Not available,Regularly Scheduled,Approximate times,66120,0,60.0
404641,38485897,07:59:00,07:59:00,10642,91,NaN,1,0,0,fhill,mbta,mbta2018_102418_20221207_1,3,Not available,Regularly Scheduled,Approximate times,28740,0,120.0


In [119]:
stop_times = gtfsfeeds_dfs.stop_times
for trip in tqdm(trip_stop_replace.keys()):
    start_time = stop_times.query('(trip_id == @trip) & (first_stop == 1)')['departure_time_sec']
    # drop old stop times
    stop_times = stop_times.drop(
        stop_times.loc[stop_times['trip_id']==trip].index)
    # grab new stop times
    new_trip = trip_stop_replace[trip]
    nst = stop_times.query('trip_id == @new_trip')
    nst['trip_id'] = trip

    # replace the start time, then calculate the stop times by the departure_time_sec difference
    nst.loc[nst.loc[:,'first_stop']==1,'departure_time_sec'] = int(start_time.iloc[0])
    nst.loc[nst.loc[:,'first_stop']==1,'time_between_stops'] = int(start_time.iloc[0])
    nst['departure_time_sec'] = nst['time_between_stops'].cumsum()

    # recalc arrival/dep times
    nst['arrival_time'] = pd.to_datetime(nst['departure_time_sec'],unit='s').astype('str').str[11:19]
    nst['departure_time'] = nst['arrival_time']

    #keep only relevant columns
    nst = nst[stop_times.columns]

    stop_times = pd.concat([stop_times,nst])

100%|██████████| 1671/1671 [16:07:23<00:00, 34.74s/it]       


In [120]:
gtfsfeeds_dfs.stop_times = stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'timepoint',
       'checkpoint_id', 'unique_agency_id', 'unique_feed_id', 'route_type',
       'pickup_type_desc', 'drop_off_type_desc', 'timepoint_desc',
       'departure_time_sec']]


In [121]:
gtfsfeeds_dfs.trips = pd.concat([gtfsfeeds_dfs.trips.drop(columns=['route_pattern_id_old','trip_headsign_old','shape_id_new']), non_bus_trips])

In [122]:
gtfsfeeds_dfs.stop_times = gtfsfeeds_dfs.stop_times.sort_values(by=['trip_id','stop_sequence'])

In [123]:
gtfsfeeds_dfs.trips.to_csv(r"C:\Users\matkinson.AD\Downloads\Dec07_Sandbox\Part_3_Py_Consolidation\mbta2018_post_py\trips.txt",index=False)
gtfsfeeds_dfs.stop_times.to_csv(r"C:\Users\matkinson.AD\Downloads\Dec07_Sandbox\Part_3_Py_Consolidation\mbta2018_post_py\stop_times.txt",index=False)

# COMPARE: TransCAD 826 to GTFS

In [ ]:
len(gtfsfeeds_dfs.stop_times['trip_id'].unique())

In [ ]:
p4 = pd.read_csv(r"C:\Users\matkinson.AD\Downloads\Nov15_Sandbox\Part_4_Import_GTFS\test_routes.csv")
p4_a = p4.groupby(by=['Route','Direction','Sign']).agg({'Route_Name':'count','Daily_Trips':'sum'}).reset_index().rename(columns={'Route':'route_id','Direction':'direction_id','Sign':'trip_headsign'})
p4_a['direction_id'] = np.where(p4_a['direction_id'] == "F", 0, 1)
p4_a

In [ ]:
trips_a = gtfsfeeds_dfs.trips.groupby(by=['route_id','direction_id','trip_headsign']).agg({'route_pattern_id':'nunique','trip_id':'count'}).rename(columns={'trip_id':'gtfs_daily_trips'}).reset_index()
trips_a

In [ ]:
trips_a.query('route_pattern_id > 1')

In [ ]:
trips_b = trips_a.merge(p4_a, how='outer',on=['route_id','direction_id','trip_headsign'])
trips_b.query('route_pattern_id != Route_Name') # where multiple route patterns per R/D/H combo


In [ ]:
trips_b.query('Daily_Trips.isna()')

In [ ]:
trips_aa = trips_a.rename(columns={'direction_id':'Direction','route_id':'Route','trip_headsign':'Sign'})
trips_aa['Direction'] = np.where(trips_aa['Direction'] == 0, "F", "R")

In [ ]:
p5 = p4.merge(trips_aa, how='outer',on=['Route','Direction','Sign']).query('Daily_Trips != gtfs_daily_trips')
p5

In [ ]:
fred = trips_b.query('route_pattern_id != Route_Name')
fred = fred[['route_id','direction_id','trip_headsign']].rename(columns={'route_id':'Route','direction_id':'Direction','trip_headsign':'Sign'})
fred['Direction'] = np.where(fred['Direction'] == 0, "F", "R")
george = fred.merge(p5, how='left',on=['Route','Direction','Sign'])
george

In [ ]:
george.sort_values('Route')[['Route','Direction','Sign','Route_Name','Long Name','Daily_Trips','gtfs_daily_trips','Length']]

In [ ]:
ginny = george.merge(gtfsfeeds_dfs.trips, how='left',left_on='Trip',right_on='trip_id')[['Route','Direction','Sign','Route_Name','Long Name','Daily_Trips','gtfs_daily_trips','Length','trip_headsign','route_pattern_id_y','trip_id']]
ginny

In [ ]:
ginny.merge(trips_keep_safe, on='trip_id',how='left')#.query('route_pattern_id_y != route_pattern_id').sort_values('Route')#.query('trip_headsign_x != trip_headsign_y')#

In [ ]:
ginny.query('Route=="10"')['trip_id'].to_list()

In [ ]:
stop_odd_10 = gtfsfeeds_dfs.stop_times.query('trip_id == "38231507" | trip_id == "38231390"')

In [ ]:
stop_odd_10